# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# Import libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic
import json
import requests

In [ ]:
# Check for API Keys and open clients

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
weather_api_key = os.getenv('WEATHER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
    openai = OpenAI()
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
    claude = anthropic.Anthropic()
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key="ollama")
else:
    print("Google API Key not set")

if weather_api_key:
    print(f"Weather API Key exists and begins {weather_api_key[:7]}")
else:
    print("Weather API Key not set")

# Set up model global constants
GPT_MODEL = 'gpt-4o-mini'
ANTHROPIC_MODEL='claude-3-haiku-20240307'
OLLAMA_MODEL = 'llama3.2' # Llama 3.2 works well with tools via the OpenAI API. Gemma3 can't use tools, IBM Granite 3 8b recognizes the tool exists, but it doesn't call it via the openai API.

Two versions of the system prompt. 
1. One for conversational specification definitions helper, 
2. one for a weather checking assistant (with tool calling)

In [ ]:
# Define system prompt

# system_message = """ You are a software specification specialist. You will be presented with an idea for a software application. Your job is to ask detailed questions to gather enough information 
# to create a specification document for this application. The questions should deal with one topic at a time and you will ask them one by one, until you gather all the information you need. 
# our final task after the conversation has concluded is to provide the specification document formatted in Markdown. 
#"""

system_message = "You are a helpful weather assistant. You will respond to questions on the current weather for a city."

In [ ]:
# Tool definitions go here
def get_weather(location: str) -> None:
    """
    Simple function to get and display current weather
    
    Args:
        location (str): Location to check weather for
    """
    url = f"http://api.weatherapi.com/v1/current.json"
    
    params = {
        'key': weather_api_key,
        'q': location,
        'aqi': 'no'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        loc = data['location']
        weather = data['current']

        #Debug printouts
       #print(f"Weather in {loc['name']}: {weather['condition']['text']}")
        #print(f"Temperature: {weather['temp_c']}°C")
        #print(f"Humidity: {weather['humidity']}%")
        
        return weather
        
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

In [ ]:
#get_weather("Munich Germany")

In [ ]:
# Tool JSON declarations
get_weather_function = {
    "name": "get_weather",
    "description": "Get the weather at a given location. Call this whenever you get asked what the weather is like in a city or place.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location that the user wants to know the weather for",
            },
        },
        "required": ["location"],
        "additionalProperties": False
    }
}

# And this is included in a list of tools:
tools = [{"type": "function", "function": get_weather_function}]

In [ ]:
# Define function to handle tool calls
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name=='get_weather' :
        arguments = json.loads(tool_call.function.arguments)
        location = arguments.get('location')
        weather = get_weather(location)
        # Keep weather condition, temperature and humidity from the results. 
        response = {
            "role": "tool",
            "content": json.dumps({"location": location,"weather": weather['condition']['text'], "temperature_celsius": weather['temp_c'], "humidity_percent": weather['humidity']}),
            "tool_call_id": tool_call.id
        }
        return response, location
    else: 
        print("Unknown tool")

## Two versions of the chat function. 

One with a tool call and the other for simple streaming chat. The streaming output complicates things as tool call data may be streamed. 

Also, Anthropic has a different API structure for tool calls so we need some helper functions to keep things neat.

*TODO*: 
1. Create the streaming tool calling version with just OpenAI API
2. Add handlers for Anthropic API tool calls
3. Add handlers for other local models (e.g. IBM Granite 3)
4. Work on a better interface (currently basic gradio ChatInterface).

In [ ]:
# Handle chat functions based on different models (streaming version, no tools). This version works with both openai and anthropic models, but doesn't use tools

def chat(message, history, dropdown_value):
    if dropdown_value=="GPT":
        messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
        stream = openai.chat.completions.create(model=GPT_MODEL, messages=messages, stream=True)
        response = ""
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            yield response
    elif dropdown_value=="Ollama":
        messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
        stream = ollama_via_openai.chat.completions.create(model=OLLAMA_MODEL, messages=messages, stream=True)
        response = ""
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            yield response
    elif dropdown_value=="Claude":
        # Keep only 'role' and 'content' keys from history.
        keys_to_keep = ['role', 'content']
        history_filtered = [{k: v for k,v in row.items() if k in keys_to_keep} for row in history]
        messages = history_filtered + [{"role":"user", "content": message}]
        print(messages)
        result = claude.messages.stream(
            model=ANTHROPIC_MODEL,
            max_tokens=1000,
            temperature=0.7,
            system=system_message,
            messages=messages
        )
        response = ""
        with result as stream:
            for text in stream.text_stream:
                response += text or ""
                yield response
    else:
        return "Error! Invalid Chatbot!"


This is the tool handling version, no streaming:

In [ ]:
# handle tool calls without streaming. This version works with tools (no Anthropic)
# TODO: Add Anthropic API tool calls
def chat_no_streaming(message, history, dropdown_value):
    if dropdown_value == "GPT":
        client = openai
        model = GPT_MODEL
    elif dropdown_value == "Ollama":
        client = ollama_via_openai
        model = OLLAMA_MODEL
    else:
        return "Invalid model selection"
    
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=model, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, location = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = client.chat.completions.create(model=model, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# Define Gradio Chat Interface. Uncomment for Claude.

gr.ChatInterface(fn=chat_no_streaming, type="messages",
                additional_inputs=[gr.Dropdown(["GPT", "Ollama"], label="Select model", value="Ollama")]).launch()

#gr.ChatInterface(fn=chat_no_streaming, type="messages",
#               additional_inputs=[gr.Dropdown(["GPT", "Ollama", "Claude"], label="Select model", value="Ollama")]).launch()